# Creating Raw Nexus File

In [26]:
import numpy as np
import h5py
from lxml import etree

# ------------------------------------------------------------------------------
# 1) CONFIGURATION
# ------------------------------------------------------------------------------
nxdl_file_path = "NXem.nxdl.xml"       # Path to your NXem definition XML
output_nexus_file = "NXem_regenerated.nxs"   # Output file name

# XML namespace for NXDL
NS = {'nx': 'http://definition.nexusformat.org/nxdl/3.1'}

# ------------------------------------------------------------------------------
# 2) PARSE THE NXDL FILE
# ------------------------------------------------------------------------------
tree = etree.parse(nxdl_file_path)
root = tree.getroot()

# ------------------------------------------------------------------------------
# 3) RECURSIVELY EXTRACT THE NXDL STRUCTURE
# ------------------------------------------------------------------------------
def extract_full_structure(parent_element):
    """
    Recursively find <group> or <field> tags in the NXDL and build a nested
    Python list/dict structure describing them.
    """
    structure = []
    for element in parent_element:
        # Skip comments and processing instructions:
        if not isinstance(element.tag, str):
            continue

        tag_name = etree.QName(element.tag).localname

        if tag_name in ["group", "field"]:
            # For groups, 'name' might be set or 'type' might be used as fallback
            # For fields, 'name' is definitely present (type can indicate NX_CHAR, etc.)
            structure.append({
                "tag": tag_name,
                "name": element.get("name") or element.get("type"),
                "type": element.get("type"),
                "children": extract_full_structure(element)  # recursion
            })
    return structure

# ------------------------------------------------------------------------------
# 4) GENERATE A DEFAULT VALUE FOR FIELDS BASED ON TYPE
# ------------------------------------------------------------------------------
def get_default_value(nx_type):
    """
    Return a simple placeholder data value for each field, based on the NXDL type.
    """
    if nx_type:
        lower = nx_type.lower()
        if "int" in lower:
            return 0
        elif "float" in lower or "double" in lower:
            return 0.0
        elif "char" in lower or "string" in lower:
            # By default, let's just say "unset"
            return np.bytes_("unset")
    # Fallback string
    return np.bytes_("unset")

# ------------------------------------------------------------------------------
# 5) FUNCTION TO WRITE HDF5
# ------------------------------------------------------------------------------
def create_nexus_file(filename, structure):
    """
    Creates an HDF5 file from a list of (group or field) dictionaries.
    """
    def add_elements(h5group, elements):
        for el in elements:
            if not el.get("tag") or not el.get("name"):
                continue

            tag = el["tag"]
            name = el["name"]
            nx_type = el["type"]
            children = el.get("children", [])

            if tag == "group":
                # Create subgroup
                subgroup = h5group.create_group(name)
                # If "type" is e.g. NXentry, store it in the NX_class attribute
                if nx_type:
                    subgroup.attrs["NX_class"] = nx_type
                # Recursively add child elements
                add_elements(subgroup, children)

            elif tag == "field":
                # Create a dataset. If we have an explicit "value" key, use that.
                # Otherwise generate a default placeholder from the type.
                if "value" in el:
                    dataset_value = np.bytes_(el["value"])
                else:
                    dataset_value = get_default_value(nx_type)

                try:
                    if name not in h5group:
                        h5group.create_dataset(name, data=dataset_value)
                    else:
                        print(f"⚠️ Skipping existing dataset '{name}'")
                except Exception as exc:
                    print(f"⚠️ Could not create dataset '{name}': {exc}")

    with h5py.File(filename, "w") as f:
        add_elements(f, structure)

# ------------------------------------------------------------------------------
# 6) FIND THE TOP-LEVEL GROUP THAT EXTENDS "NXentry"
# ------------------------------------------------------------------------------
entry_groups = root.findall(".//nx:group[@type='NXentry']", namespaces=NS)
if not entry_groups:
    raise ValueError("❌ No <group type='NXentry'> found in NXDL XML!")

# Extract the structure under this NXentry block
entry_structure = extract_full_structure(entry_groups[0])

# ------------------------------------------------------------------------------
# 7) MANUALLY INSERT A 'definition' FIELD = "NXem"
# ------------------------------------------------------------------------------
# By explicitly setting "value": "NXem" here, we ensure the dataset is exactly "NXem".
# That prevents NOMAD from searching for "NXem.nxdl.xml.nxdl.xml" or similar.
entry_structure.insert(0, {
    "tag": "field",
    "name": "definition",
    "type": "NX_CHAR",
    "value": "NXem"   # <--- crucial so that NOMAD sees the definition as "NXem"
})

# ------------------------------------------------------------------------------
# 8) WRAP THIS ENTRY STRUCTURE IN A TOP-LEVEL GROUP NAMED "/entry" with NX_class = NXentry
# ------------------------------------------------------------------------------
wrapped_structure = [{
    "tag": "group",
    "name": "entry",
    "type": "NXentry",
    "children": entry_structure
}]

# ------------------------------------------------------------------------------
# 9) WRITE THE NeXus FILE
# ------------------------------------------------------------------------------
create_nexus_file(output_nexus_file, wrapped_structure)
print(f"✅ NeXus file created: {output_nexus_file}")


⚠️ Skipping existing dataset 'definition'
✅ NeXus file created: NXem_regenerated.nxs


In [27]:
import h5py

with h5py.File("NXem_regenerated.nxs", "r") as f:
    f.visit(lambda name: print(name))

entry
entry/NXcoordinate_system_set
entry/NXcoordinate_system_set/TRANSFORMATIONS
entry/NXdata
entry/NXmonitor
entry/NXprogram
entry/NXprogram/program
entry/NXuser
entry/NXuser/address
entry/NXuser/affiliation
entry/NXuser/email
entry/NXuser/name
entry/NXuser/orcid
entry/NXuser/orcid_platform
entry/NXuser/role
entry/NXuser/social_media_name
entry/NXuser/social_media_platform
entry/NXuser/telephone_number
entry/definition
entry/em_lab
entry/em_lab/DETECTOR
entry/em_lab/DETECTOR/NXfabrication
entry/em_lab/DETECTOR/NXfabrication/identifier
entry/em_lab/DETECTOR/local_name
entry/em_lab/EBEAM_DEFLECTOR
entry/em_lab/EBEAM_DEFLECTOR/NXfabrication
entry/em_lab/EBEAM_DEFLECTOR/NXfabrication/identifier
entry/em_lab/EBEAM_DEFLECTOR/NXfabrication/model
entry/em_lab/EBEAM_DEFLECTOR/NXfabrication/vendor
entry/em_lab/EBEAM_DEFLECTOR/pixel_time
entry/em_lab/IBEAM_DEFLECTOR
entry/em_lab/IBEAM_DEFLECTOR/NXfabrication
entry/em_lab/IBEAM_DEFLECTOR/NXfabrication/identifier
entry/em_lab/IBEAM_DEFLECTOR/NXfa

In [28]:
from nexusformat.nexus import NXFile

with NXFile("NXem_regenerated.nxs", "r") as f:
    nxroot = f.readfile()
    print(nxroot.tree)

root:NXroot
  entry:NXentry
    NXcoordinate_system_set:NXcoordinate_system_set
      TRANSFORMATIONS:NXtransformations
    NXdata:NXdata
    NXmonitor:NXmonitor
    NXprogram:NXprogram
      program = 'unset'
    NXuser:NXuser
      address = 'unset'
      affiliation = 'unset'
      email = 'unset'
      name = 'unset'
      orcid = 'unset'
      orcid_platform = 'unset'
      role = 'unset'
      social_media_name = 'unset'
      social_media_platform = 'unset'
      telephone_number = 'unset'
    definition = 'NXem'
    em_lab:NXinstrument
      DETECTOR:NXdetector
        NXfabrication:NXfabrication
          identifier = 'unset'
        local_name = 'unset'
      EBEAM_DEFLECTOR:NXscanbox_em
        NXfabrication:NXfabrication
          identifier = 'unset'
          model = 'unset'
          vendor = 'unset'
        pixel_time = 0.0
      IBEAM_DEFLECTOR:NXscanbox_em
        NXfabrication:NXfabrication
          identifier = 'unset'
          model = 'unset'
          vendor = '

# Reading+Mapping Metadata of Tescan Amber X

In [5]:
import os
import h5py
import json
import numpy as np
import tifffile
import imageio.v3 as iio
from configparser import ConfigParser

# --- CONFIGURATION ---
INPUT_IMAGE_FILE = "tescan_image.png"        # Could also be .tif or .tiff
OUTPUT_NEXUS_FILE = "tescan_output.nxs"

# --- MAPPING: Tescan metadata keys => NeXus paths ---
mapping = {
  # ––– MAIN –––
  "MAIN.AccFrames":    "entry/NXmonitor/value",
  "MAIN.AccType":      None,
  "MAIN.Company":      "entry/em_lab/NXfabrication/vendor",
  "MAIN.Date":         "entry/start_time",
  "MAIN.Description":  "entry/em_lab/NXebeam_column/NXaperture_em/description",
  "MAIN.Device":       "entry/em_lab/instrument_name",
  "MAIN.DeviceModel":  "entry/em_lab/NXfabrication/model",
  "MAIN.FullUserName": "entry/NXuser/name",
  "MAIN.ImageStripSize": None,
  "MAIN.Magnification":          "entry/em_lab/NXoptical_system_em/magnification",
  "MAIN.MagnificationReference": "entry/em_lab/NXebeam_column/aberration_correction/ZEMLIN_TABLEAU/magnification",
  "MAIN.OrigFileName":  None,
  "MAIN.PixelSizeX":    "entry/measurement/NXevent_data_em/IMAGE_SET/stack/axis_x",
  "MAIN.PixelSizeY":    "entry/measurement/NXevent_data_em/IMAGE_SET/stack/axis_y",
  "MAIN.SerialNumber":  "entry/em_lab/NXfabrication/identifier",
  "MAIN.Sign":          None,
  "MAIN.SoftwareVersion":"entry/NXprogram/program",
  "MAIN.Time":          "entry/start_time",
  "MAIN.UserName":      "entry/NXuser/name",
  "MAIN.ViewFieldsCountX": None,
  "MAIN.ViewFieldsCountY": None,

  # ––– FIB –––
  "FIB.ChamberPressure":   "entry/em_lab/NXchamber/pressure",
  "FIB.ColumnPressure":    "entry/em_lab/NXebeam_column/NXchamber/pressure",
  "FIB.ColumnTilt":        "entry/em_lab/NXstage_lab/tilt_1",
  "FIB.ColumnType":        None,
  "FIB.Condenser":         "entry/em_lab/NXebeam_column/NXlens_em/current",
  "FIB.Detector":          "entry/em_lab/DETECTOR/local_name",
  "FIB.Detector0":         "entry/em_lab/DETECTOR/local_name",
  "FIB.Detector0FlatField": None,
  "FIB.Detector0Gain":      None,
  "FIB.Detector0Offset":    None,
  "FIB.DwellTime":          "entry/em_lab/EBEAM_DEFLECTOR/pixel_time",
  "FIB.EmissionCurrent":    "entry/em_lab/NXibeam_column/ion_source/current",
  "FIB.Extractor":          "entry/em_lab/NXibeam_column/ion_source/voltage",
  "FIB.GunPressure":        None,
  "FIB.HV":                 "entry/em_lab/NXebeam_column/electron_source/voltage",
  "FIB.ImageShiftX":        None,
  "FIB.ImageShiftY":        None,
  "FIB.LUTGamma":           None,
  "FIB.LUTMaximum":         None,
  "FIB.LUTMinimum":         None,
  "FIB.LastPreset":         None,
  "FIB.Objective":          "entry/em_lab/NXebeam_column/NXlens_em/value",
  "FIB.PredictedBeamCurrent": None,
  "FIB.PresetAlignmentX":   None,
  "FIB.PresetAlignmentY":   None,
  "FIB.PrimaryDetectorGain": None,
  "FIB.PrimaryDetectorOffset": None,
  "FIB.ProbeApertureDiameter": "entry/em_lab/NXebeam_column/NXaperture_em/value",
  "FIB.ProbeApertureIndex":    "entry/em_lab/NXebeam_column/NXaperture_em/name",
  "FIB.ProbeApertureLabel":    "entry/em_lab/NXebeam_column/NXaperture_em/name",
  "FIB.ProbeAperturePositionX": None,
  "FIB.ProbeAperturePositionY": None,
  "FIB.ScanID":               "entry/measurement/NXevent_data_em/event_identifier",
  "FIB.ScanRotation":         "entry/em_lab/NXstage_lab/rotation",
  "FIB.ScanSpeed":            None,
  "FIB.SessionID":            "entry/measurement/NXevent_data_em/event_identifier",
  "FIB.SpotSize":             None,
  "FIB.StageRotation":        "entry/em_lab/NXstage_lab/rotation",
  "FIB.StageTilt":            "entry/em_lab/NXstage_lab/tilt_1",
  "FIB.StageX":               "entry/em_lab/NXstage_lab/position",
  "FIB.StageY":               "entry/em_lab/NXstage_lab/position",
  "FIB.StageZ":               "entry/em_lab/NXstage_lab/position",
  "FIB.StigmatorX":           None,
  "FIB.StigmatorY":           None,
  "FIB.TiltCorrection":       "entry/em_lab/NXebeam_column/aberration_correction/tilt_angle",
  "FIB.WD":                   "entry/em_lab/NXoptical_system_em/working_distance"
}
# ✅ Fix: replace dots with slashes in NeXus paths
# MAPPING_DICT = {k: v.replace('.', '/') for k, v in json.loads(MAPPING_JSON).items()}

# --- PARSE Tescan INI-style METADATA ---
def parse_ini_metadata(ini_text):
    parser = ConfigParser()
    parser.optionxform = str  # preserve case
    parser.read_string(ini_text)

    metadata = {}
    for section in parser.sections():
        for key, val in parser.items(section):
            full_key = f"{section}.{key}"
            metadata[full_key] = val
    return metadata

# --- ENSURE GROUP EXISTS ---
def ensure_hdf5_group(h5file, path):
    parts = path.strip('/').split('/')
    grp = h5file
    for part in parts:
        if part not in grp:
            grp = grp.create_group(part)
        else:
            grp = grp[part]
    return grp

# --- READ IMAGE + METADATA (.tif or .png+.hdr) ---
def read_image_and_metadata(filepath):
    ext = os.path.splitext(filepath)[-1].lower()
    base = os.path.splitext(filepath)[0]

    if ext in ['.tif', '.tiff']:
        with tifffile.TiffFile(filepath) as tif:
            page = tif.pages[0]
            image_data = page.asarray()
            metadata_dict = {}

            desc_tag = page.tags.get("ImageDescription")
            if desc_tag:
                ini_text = desc_tag.value
                metadata_dict = parse_ini_metadata(ini_text)

        return image_data, metadata_dict

    elif ext == '.png':
        image_data = iio.imread(filepath)

        hdr_path = base + ".hdr"
        if not os.path.exists(hdr_path):
            raise FileNotFoundError(f"No metadata file found at: {hdr_path}")

        with open(hdr_path, "r", encoding="utf-8") as f:
            ini_text = f.read()
        metadata_dict = parse_ini_metadata(ini_text)

        return image_data, metadata_dict

    else:
        raise ValueError(f"Unsupported image format: {ext}")


In [6]:
image, metadata = read_image_and_metadata(INPUT_IMAGE_FILE)
print("✅ Loaded image with shape:", image.shape)
print("✅ Parsed metadata keys (sample):", list(metadata.keys())[:])
print("✅ Total metadata entries:", len(metadata))

✅ Loaded image with shape: (1104, 1024)
✅ Parsed metadata keys (sample): ['MAIN.AccFrames', 'MAIN.AccType', 'MAIN.Company', 'MAIN.Date', 'MAIN.Description', 'MAIN.Device', 'MAIN.DeviceModel', 'MAIN.FullUserName', 'MAIN.ImageStripSize', 'MAIN.Magnification', 'MAIN.MagnificationReference', 'MAIN.OrigFileName', 'MAIN.PixelSizeX', 'MAIN.PixelSizeY', 'MAIN.SerialNumber', 'MAIN.Sign', 'MAIN.SoftwareVersion', 'MAIN.Time', 'MAIN.UserName', 'MAIN.ViewFieldsCountX', 'MAIN.ViewFieldsCountY', 'FIB.ChamberPressure', 'FIB.ColumnPressure', 'FIB.ColumnTilt', 'FIB.ColumnType', 'FIB.Condenser', 'FIB.Detector', 'FIB.Detector0', 'FIB.Detector0FlatField', 'FIB.Detector0Gain', 'FIB.Detector0Offset', 'FIB.DwellTime', 'FIB.EmissionCurrent', 'FIB.Extractor', 'FIB.GunPressure', 'FIB.HV', 'FIB.ImageShiftX', 'FIB.ImageShiftY', 'FIB.LUTGamma', 'FIB.LUTMaximum', 'FIB.LUTMinimum', 'FIB.LastPreset', 'FIB.Objective', 'FIB.PredictedBeamCurrent', 'FIB.PresetAlignmentX', 'FIB.PresetAlignmentY', 'FIB.PrimaryDetectorGain',

# Combining two Scripts
(creating Nexus file from data and metadata of Tescan)

In [8]:
"""
tescan_to_nexus.py
-------------------------------------------
Build blank NXem, read Tescan image + metadata,
write both into a NeXus file without size limits.
"""

# ───────── paths ─────────
NXDL_FILE         = "NXem.nxdl.xml"
INPUT_IMAGE_FILE  = "tescan_image.png"   # .png(+hdr) or .tif
OUTPUT_NEXUS_FILE = "tescan_output.nxs"

# ───────── imports ─────────
import os, numpy as np, h5py, tifffile, imageio.v3 as iio
from lxml import etree
from configparser import ConfigParser

# ───────── mapping (unchanged) ─────────
mapping = {
  "MAIN.AccFrames":                 "entry/NXmonitor/value",
  "MAIN.AccType":                   None,
  "MAIN.Company":                   "entry/em_lab/NXfabrication/vendor",
  "MAIN.Date":                      "entry/start_time",
  "MAIN.Description":               "entry/em_lab/NXebeam_column/NXaperture_em/description",
  "MAIN.Device":                    "entry/em_lab/instrument_name",
  "MAIN.DeviceModel":               "entry/em_lab/NXfabrication/model",
  "MAIN.FullUserName":              "entry/NXuser/name",
  "MAIN.ImageStripSize":            None,
  "MAIN.Magnification":             "entry/em_lab/NXoptical_system_em/magnification",
  "MAIN.MagnificationReference":    "entry/em_lab/NXebeam_column/aberration_correction/ZEMLIN_TABLEAU/magnification",
  "MAIN.OrigFileName":              None,
  "MAIN.PixelSizeX":                "entry/measurement/NXevent_data_em/IMAGE_SET/stack/axis_x",
  "MAIN.PixelSizeY":                "entry/measurement/NXevent_data_em/IMAGE_SET/stack/axis_y",
  "MAIN.SerialNumber":              "entry/em_lab/NXfabrication/identifier",
  "MAIN.SoftwareVersion":           "entry/NXprogram/program",
  "MAIN.Time":                      "entry/start_time",
  "MAIN.UserName":                  "entry/NXuser/name",

  "FIB.ChamberPressure":            "entry/em_lab/NXchamber/pressure",
  "FIB.ColumnPressure":             "entry/em_lab/NXebeam_column/NXchamber/pressure",
  "FIB.ColumnTilt":                 "entry/em_lab/NXstage_lab/tilt_1",
  "FIB.Condenser":                  "entry/em_lab/NXebeam_column/NXlens_em/current",
  "FIB.Detector":                   "entry/em_lab/DETECTOR/local_name",
  "FIB.DwellTime":                  "entry/em_lab/EBEAM_DEFLECTOR/pixel_time",
  "FIB.EmissionCurrent":            "entry/em_lab/NXibeam_column/ion_source/current",
  "FIB.Extractor":                  "entry/em_lab/NXibeam_column/ion_source/voltage",
  "FIB.HV":                         "entry/em_lab/NXebeam_column/electron_source/voltage",
  "FIB.Objective":                  "entry/em_lab/NXebeam_column/NXlens_em/value",
  "FIB.ProbeApertureDiameter":      "entry/em_lab/NXebeam_column/NXaperture_em/value",
  "FIB.ProbeApertureIndex":         "entry/em_lab/NXebeam_column/NXaperture_em/name",
  "FIB.ScanID":                     "entry/measurement/NXevent_data_em/event_identifier",
  "FIB.ScanRotation":               "entry/em_lab/NXstage_lab/rotation",
  "FIB.SessionID":                  "entry/measurement/NXevent_data_em/event_identifier",
  "FIB.StageRotation":              "entry/em_lab/NXstage_lab/rotation",
  "FIB.StageTilt":                  "entry/em_lab/NXstage_lab/tilt_1",
  "FIB.StageX":                     "entry/em_lab/NXstage_lab/position",
  "FIB.StageY":                     "entry/em_lab/NXstage_lab/position",
  "FIB.StageZ":                     "entry/em_lab/NXstage_lab/position",
  "FIB.TiltCorrection":             "entry/em_lab/NXebeam_column/aberration_correction/tilt_angle",
  "FIB.WD":                         "entry/em_lab/NXoptical_system_em/working_distance"
}


# ───────── helpers ─────────
def parse_ini(txt:str)->dict:
    cp=ConfigParser(); cp.optionxform=str; cp.read_string(txt)
    return {f"{s}.{k}":v for s in cp.sections() for k,v in cp.items(s)}

def ensure_grp(h: h5py.Group, p:str)->h5py.Group:
    g=h
    for part in p.strip("/").split("/"):
        g=g.require_group(part)
    return g

# ───────── 1. safer blank builder ─────────
def build_blank(nxdl:str, outnxs:str):
    ns={"nx":"http://definition.nexusformat.org/nxdl/3.1"}
    root=etree.parse(nxdl).getroot()
    entry=root.find(".//nx:group[@type='NXentry']",ns)
    if entry is None: raise RuntimeError("NXentry not found")

    # collect all nested <group> nodes
    groups=[]
    def visit(node):
        for el in node:
            if isinstance(el.tag,str) and etree.QName(el.tag).localname=="group":
                groups.append(el); visit(el)
    visit(entry)

    with h5py.File(outnxs,"w") as f:
        ent=ensure_grp(f,"/entry"); ent.attrs["NX_class"]="NXentry"
        ent.create_dataset("definition",
                           data=np.bytes_("NXem"),
                           dtype=h5py.string_dtype("utf-8"))

        for g in groups:
            # build path components up to NXentry
            comps=[]
            n=g
            while n is not None and n is not entry:
                comps.append(n.get("name") or n.get("type"))
                n=n.getparent()
            comps.reverse()
            grp=ensure_grp(f,"/entry/" + "/".join(comps))
            if g.get("type"):
                grp.attrs["NX_class"]=g.get("type")

# ───────── 2. read Tescan (unchanged) ─────────
def read_tescan(path:str):
    base,ext=os.path.splitext(path); ext=ext.lower()
    if ext in(".tif",".tiff"):
        with tifffile.TiffFile(path) as tf:
            img=tf.pages[0].asarray()
            meta={}
            tag=tf.pages[0].tags.get("ImageDescription")
            if tag: meta.update(parse_ini(tag.value))
        return img,meta
    if ext==".png":
        img=iio.imread(path)
        hdr=base+".hdr"
        if not os.path.exists(hdr): raise FileNotFoundError(hdr)
        return img,parse_ini(open(hdr,encoding="utf-8").read())
    raise ValueError("unsupported")

# ───────── scalar helpers ─────────
def to_scalar(v:str):
    try:
        f=float(v); return int(f) if f.is_integer() else f
    except Exception: return v.encode()

def write_scalar(parent: h5py.Group,name:str,val):
    if name in parent:
        ds=parent[name]
        need_new=False
        if isinstance(val,(bytes,str)):
            if not h5py.check_string_dtype(ds.dtype): need_new=True
            elif h5py.check_string_dtype(ds.dtype).length and \
                 len(val)>ds.dtype.itemsize:         need_new=True
        else:
            if h5py.check_string_dtype(ds.dtype):    need_new=True
        if need_new:
            del parent[name]
            return write_scalar(parent,name,val)
        ds[...]=val
    else:
        if isinstance(val,(bytes,str)):
            parent.create_dataset(name,data=val,dtype=h5py.string_dtype("utf-8"))
        else:
            parent.create_dataset(name,data=val)

# ───────── 3. fill file (unchanged logic) ─────────
def fill_nexus(nxs:str,img,npmeta,mp):
    with h5py.File(nxs,"a") as f:
        nxdata=ensure_grp(f,"/entry/data"); nxdata.attrs.update({"NX_class":"NXdata","signal":"data"})
        if "data" in nxdata: nxdata["data"][...]=img
        else: nxdata.create_dataset("data",data=img)

        for k,p in mp.items():
            if p is None or k not in npmeta: continue
            parent_path,dname=os.path.split(p)
            parent=ensure_grp(f,parent_path)
            if dname in parent and isinstance(parent[dname],h5py.Group):
                parent=parent[dname]; dname="value"
            write_scalar(parent,dname,to_scalar(npmeta[k]))

# ───────── main ─────────
if __name__=="__main__":
    build_blank(NXDL_FILE,OUTPUT_NEXUS_FILE)
    print("blank NXem built")

    img,meta=read_tescan(INPUT_IMAGE_FILE)
    print("read Tescan:",img.shape,"meta",len(meta))

    fill_nexus(OUTPUT_NEXUS_FILE,img,meta,mapping)
    print("done ->",OUTPUT_NEXUS_FILE)


blank NXem built
read Tescan: (1104, 1024) meta 67
done -> tescan_output.nxs


In [9]:
from nexusformat.nexus import NXFile

with NXFile("tescan_output.nxs", "r") as f:
    nxroot = f.readfile()
    print(nxroot.tree)

root:NXroot
  entry:NXentry
    NXcoordinate_system_set:NXcoordinate_system_set
      TRANSFORMATIONS:NXtransformations
    NXdata:NXdata
    NXmonitor:NXmonitor
      value = 2
    NXprogram:NXprogram
      program = 'TESCAN Essence Version 1.3.1.0, build 7450'
    NXuser:NXuser
      name = 'LAME_Supervisor'
    data:NXdata
      @signal = 'data'
      data = uint16(1104x1024)
    definition = 'NXem'
    em_lab:NXinstrument
      DETECTOR:NXdetector
        NXfabrication:NXfabrication
        local_name = 'SE'
      EBEAM_DEFLECTOR:NXscanbox_em
        NXfabrication:NXfabrication
        pixel_time = 1e-06
      IBEAM_DEFLECTOR:NXscanbox_em
        NXfabrication:NXfabrication
      NXchamber:NXchamber
        pressure = 0.00162483
      NXebeam_column:NXebeam_column
        NXaperture_em:NXaperture_em
          NXfabrication:NXfabrication
          description = ''
          name = 16
          value = 5e-05
        NXchamber:NXchamber
          pressure = 0.00011
        NXfabricati